In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np

tf.enable_eager_execution()

from preprocessing import inception_preprocessing
from riptide.data import *
import functools

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
preprocess = functools.partial(inception_preprocessing.preprocess_image, height=224, width=224, is_training=False)

In [61]:
test_dataset = tfrecord_dataset("/data3/imagenet/tfrecords/", "val", preprocess=preprocess, batch_size=128)

In [62]:
with tf.device("/gpu:0"):
    test_iterator = tfe.Iterator(test_dataset)

In [63]:
from tensorflow.python.keras.applications.mobilenet import MobileNet
with tf.device("/gpu:0"):
    model = MobileNet(weights='imagenet')

In [66]:
def evaluate(model, iterator, k=1):
    metric = tfe.metrics.Mean()
    for i, (data, label) in enumerate(iterator):
        with tf.device("/gpu:0"):
            z = model(data)
        metric(tf.cast(tf.nn.in_top_k(z.cpu(), tf.squeeze(label.cpu())-1, k=5), tf.float32))
        if i%10 == 0:
            print(i)
    print(metric.value())

In [ ]:
evaluate(model, test_iterator)

0
10
20
